### Imports

In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import sys

# 1. Data Exploration

### Load metadata

In [54]:
df_meta = pd.read_csv('data/train_small-db.meta',delimiter='\t')

In [55]:
df_meta

,sequence.id,genome.id,taxid.strain,taxid.species,taxid.genus
0,Cupriavidus_necator_N-1,NC_015723,1042878,106590,106589
1,Cupriavidus_necator_N-1,NC_015724,1042878,106590,106589
2,Cupriavidus_necator_N-1,NC_015726,1042878,106590,106589
3,Cupriavidus_necator_N-1,NC_015727,1042878,106590,106589
4,Rhodospirillum_rubrum_ATCC_11170,NC_007641,269796,1085,1081
...,...,...,...,...,...
1559,Burkholderia_cenocepacia_J2315,NC_011002,216591,95486,32008
1560,Burkholderia_cenocepacia_J2315,NC_011003,216591,95486,32008
1561,Burkholderia_cenocepacia_MC0-3,NC_010508,406425,95486,32008
1562,Burkholderia_cenocepacia_MC0-3,NC_010512,406425,95486,32008


### Explore metadata

In [56]:
print('Number of sequence ids:',len(df_meta['sequence.id'].unique()))
print('Number of genome ids:',len(df_meta['genome.id'].unique()))
print('Number of taxid strains:',len(df_meta['taxid.strain'].unique()))
print('Number of taxid species:',len(df_meta['taxid.species'].unique()))
print('Number of taxid genus:',len(df_meta['taxid.genus'].unique()))

Number of sequence ids: 872
Number of genome ids: 1564
Number of taxid strains: 872
Number of taxid species: 193
Number of taxid genus: 76


### Load taxid data

In [57]:
df_taxid = pd.read_csv('data/train_small-db.species-level.taxid',header = None)

In [58]:
df_taxid

,0
0,106590
1,106590
2,106590
3,106590
4,1085
...,...
1559,95486
1560,95486
1561,95486
1562,95486


### Load sequence data

In [59]:
%%time

filename='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'

count = 0
for seq_record in SeqIO.parse(filename, "fasta"):
    count += 1

count

CPU times: user 14.8 s, sys: 2.12 s, total: 16.9 s
Wall time: 17.1 s


1564

# 2. Build toy dataset

### Calculate sequence lengths
To identify small sequences which can be used to generate a toy dataset.

In [60]:
%%time

# create dataframe containing sequence lengths for each sequence
filename='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'

ids = []
lengths = []

for i,seq_record in enumerate(SeqIO.parse(filename, "fasta")):
        ids.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
df_lengths = pd.DataFrame(list(zip(ids, lengths)), columns =['genome.id', 'sequence.length'])
df_lengths.head()

CPU times: user 14.7 s, sys: 1.73 s, total: 16.4 s
Wall time: 16.5 s


,genome.id,sequence.length
0,NC_015723,2684606
1,NC_015724,424140
2,NC_015726,3872936
3,NC_015727,1499175
4,NC_007641,53732


In [61]:
# join with taxonomy information to get species
df = df_meta.set_index('genome.id').join(
    df_lengths.set_index('genome.id'),lsuffix='_caller', rsuffix='_other'
).reset_index()

In [62]:
df.head()

,genome.id,sequence.id,taxid.strain,taxid.species,taxid.genus,sequence.length
0,NC_015723,Cupriavidus_necator_N-1,1042878,106590,106589,2684606
1,NC_015724,Cupriavidus_necator_N-1,1042878,106590,106589,424140
2,NC_015726,Cupriavidus_necator_N-1,1042878,106590,106589,3872936
3,NC_015727,Cupriavidus_necator_N-1,1042878,106590,106589,1499175
4,NC_007641,Rhodospirillum_rubrum_ATCC_11170,269796,1085,1081,53732


In [63]:
# df.to_csv('data/train_small-db.lengths.csv')

In [64]:
# bin data v1
bins = [0, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10]
df['binned'] = pd.cut(df['sequence.length'], bins)

In [65]:
df['binned'].value_counts()

(1000000.0, 10000000.0]          953
(10000.0, 100000.0]              291
(1000.0, 10000.0]                198
(100000.0, 1000000.0]            122
(1000000000.0, 10000000000.0]      0
(100000000.0, 1000000000.0]        0
(10000000.0, 100000000.0]          0
(0.0, 1000.0]                      0
Name: binned, dtype: int64

In [66]:
# bin data v2
bins = [0, 1e3, 2*1e3, 5*1e3, 1e4, 1e5, 1e6, 1e7]
df['binned'] = pd.cut(df['sequence.length'], bins)

In [67]:
df['binned'].value_counts()

(1000000.0, 10000000.0]    953
(10000.0, 100000.0]        291
(5000.0, 10000.0]          134
(100000.0, 1000000.0]      122
(2000.0, 5000.0]            59
(1000.0, 2000.0]             5
(0.0, 1000.0]                0
Name: binned, dtype: int64

In [68]:
df[df['sequence.length'] < 2000]

,genome.id,sequence.id,taxid.strain,taxid.species,taxid.genus,sequence.length,binned
36,NC_013451,Staphylococcus_aureus_subsp._aureus_ED98,681288,1280,1279,1442,"(1000.0, 2000.0]"
540,NC_006375,Lactobacillus_plantarum_WCFS1,220668,1590,1578,1917,"(1000.0, 2000.0]"
757,NC_019565,Helicobacter_pylori_Aklavik86,1055532,210,209,1634,"(1000.0, 2000.0]"
818,NC_015407,Mycoplasma_mycoides_subsp._capri_LC_str._95010,862259,2102,2093,1840,"(1000.0, 2000.0]"
1197,NC_016841,Klebsiella_pneumoniae_subsp._pneumoniae_HS11286,1125630,573,570,1308,"(1000.0, 2000.0]"


In [69]:
# get sequences with lengths between 1000 and 2000 bp
genome_ids = df[df['sequence.length'] < 2000]['genome.id'].to_list()

In [70]:
genome_ids

['NC_013451', 'NC_006375', 'NC_019565', 'NC_015407', 'NC_016841']

### Select specific samples
Select smallest length samples to use for method development.

In [71]:
input_file='/Users/ryanqnelson/Downloads/large-scale-metagenomics-1.0/data/train-dataset/train_small-db.fasta'
output_file = "data/train_small-db_toy-2000.fasta"

# get toy sample sequences
sequences = []
for seq_record in SeqIO.parse(input_file, "fasta"):
    if seq_record.id in genome_ids:
        print(seq_record.id, len(seq_record.seq))
        sequences.append(seq_record)
        
# write toy sample to file
with open(output_file, "w") as output_handle:
    SeqIO.write(sequences, output_handle, "fasta")
    

NC_013451 1442
NC_006375 1917
NC_019565 1634
NC_015407 1840
NC_016841 1308


In [74]:
# verify this worked by reading results back in
input_file = "data/train_small-db_toy-2000.fasta"
for seq_record in SeqIO.parse(input_file, "fasta"):
    print(seq_record.id, len(seq_record.seq))

NC_013451 1442
NC_006375 1917
NC_019565 1634
NC_015407 1840
NC_016841 1308


### Save matching taxid.species

In [82]:
genome_ids = df[df['sequence.length'] < 2000]['taxid.species'].to_list()
print(genome_ids)
output_file = "data/train_small-db_toy-2000.taxid"
with open(output_file,'w') as output_handle:
    for each in genome_ids:
        output_handle.write(str(each) + '\n')

[1280, 1590, 210, 2102, 573]


In [13]:


print(isdir) 

True


In [18]:
with open('data/test4.npy', 'wb') as f:
    np.save(f, np.array([]))

In [11]:
np.load('data/test.npy')

array([], dtype=float64)